In [5]:
!pip install pyyaml
!pip install pandas

  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [19]:
import requests
import json
import datetime
import time
import yaml
import pandas as pd


In [34]:
with open('./config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
# CANO = _cfg['CANO']
# ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
# DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [35]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type": "application/json"}
    body = {"grant_type": "client_credentials", "appkey": APP_KEY, "appsecret": APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

In [39]:
from datetime import datetime
today = datetime.today()

def get_current_price(code=""):
    """일봉 조회"""
    PATH = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J", # 시장 분류 코드 
    "fid_input_iscd":code, # 종목 코드
#     "fid_input_date_1":"20240301", # 시작 날짜
#     "fid_input_date_2":"20240321",#today.strftime("%Y%m%d"), # 종료 날짜
    "fid_period_div_code": "W", # 기간분류코드 	D:일봉, W:주봉, M:월봉, Y:년봉
    "fid_org_adj_prc":"0"
    }
    res = requests.get(URL, headers=headers, params=params)
    
    return res.json()#['output']#['stck_bsop_date']


In [40]:
ACCESS_TOKEN = get_access_token()

In [41]:
current_price  = get_current_price("005930")
print(current_price,end='\n')

{'output': {'iscd_stat_cls_code': '55', 'marg_rate': '20.00', 'rprs_mrkt_kor_name': 'KOSPI200', 'new_hgpr_lwpr_cls_code': '신고가', 'bstp_kor_isnm': '전기.전자', 'temp_stop_yn': 'N', 'oprc_rang_cont_yn': 'N', 'clpr_rang_cont_yn': 'N', 'crdt_able_yn': 'Y', 'grmn_rate_cls_code': '40', 'elw_pblc_yn': 'Y', 'stck_prpr': '82400', 'prdy_vrss': '1600', 'prdy_vrss_sign': '2', 'prdy_ctrt': '1.98', 'acml_tr_pbmn': '2224901311700', 'acml_vol': '27126366', 'prdy_vrss_vol_rate': '108.14', 'stck_oprc': '81200', 'stck_hgpr': '82500', 'stck_lwpr': '80900', 'stck_mxpr': '105000', 'stck_llam': '56600', 'stck_sdpr': '80800', 'wghn_avrg_stck_prc': '82015.06', 'hts_frgn_ehrt': '55.31', 'frgn_ntby_qty': '7800616', 'pgtr_ntby_qty': '3701573', 'pvt_scnd_dmrs_prc': '82133', 'pvt_frst_dmrs_prc': '81466', 'pvt_pont_val': '80333', 'pvt_frst_dmsp_prc': '79666', 'pvt_scnd_dmsp_prc': '78533', 'dmrs_val': '81800', 'dmsp_val': '80000', 'cpfn': '7780', 'rstc_wdth_prc': '24200', 'stck_fcam': '100', 'stck_sspr': '63020', 'aspr_u

In [42]:
import pandas as pd


In [46]:
# 전체 데이터를 보여주는 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df = pd.DataFrame(current_price)

df = df.T

df.head()

,iscd_stat_cls_code,marg_rate,rprs_mrkt_kor_name,new_hgpr_lwpr_cls_code,bstp_kor_isnm,temp_stop_yn,oprc_rang_cont_yn,clpr_rang_cont_yn,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,stck_oprc,stck_hgpr,stck_lwpr,stck_mxpr,stck_llam,stck_sdpr,wghn_avrg_stck_prc,hts_frgn_ehrt,frgn_ntby_qty,pgtr_ntby_qty,pvt_scnd_dmrs_prc,pvt_frst_dmrs_prc,pvt_pont_val,pvt_frst_dmsp_prc,pvt_scnd_dmsp_prc,dmrs_val,dmsp_val,cpfn,rstc_wdth_prc,stck_fcam,stck_sspr,aspr_unit,hts_deal_qty_unit_val,lstn_stcn,hts_avls,per,pbr,stac_month,vol_tnrt,eps,bps,d250_hgpr,d250_hgpr_date,d250_hgpr_vrss_prpr_rate,d250_lwpr,d250_lwpr_date,d250_lwpr_vrss_prpr_rate,stck_dryy_hgpr,dryy_hgpr_vrss_prpr_rate,dryy_hgpr_date,stck_dryy_lwpr,dryy_lwpr_vrss_prpr_rate,dryy_lwpr_date,w52_hgpr,w52_hgpr_vrss_prpr_ctrt,w52_hgpr_date,w52_lwpr,w52_lwpr_vrss_prpr_ctrt,w52_lwpr_date,whol_loan_rmnd_rate,ssts_yn,stck_shrn_iscd,fcam_cnnm,cpfn_cnnm,frgn_hldn_qty,vi_cls_code,ovtm_vi_cls_code,last_ssts_cntg_qty,invt_caful_yn,mrkt_warn_cls_code,short_over_yn,sltr_yn
output,55,20.00,KOSPI200,신고가,전기.전자,N,N,N,Y,40,Y,82400,1600,2,1.98,2224901311700,27126366,108.14,81200,82500,80900,105000,56600,80800,82015.06,55.31,7800616,3701573,82133,81466,80333,79666,78533,81800,80000,7780,24200,100,63020,100,1,5969782550,4919101,38.67,1.58,12,0.45,2131.00,52002.00,82500,20240329,-0.12,62000,20230327,32.90,82500,-0.12,20240329,70700,16.55,20240118,82500,-0.12,20240329,62200,32.48,20230329,0.11,N,005930,100,"7,780 억",3302158601,N,N,39325,N,00,N,N
rt_cd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
msg_cd,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000,MCA00000
msg1,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.,정상처리 되었습니다.
